In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'

Theta_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/Alpha_time.npz'



# RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
# X = RAW['X'].astype('float64')
# Y = RAW['Y'].astype('float64')
# group = RAW['Group'].astype('float64')

# CNN-LSTM

In [8]:
%%capture
!pip install tensorflow_addons

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Alpha/CNN_LSTM/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Alpha/CNN_LSTM/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [10]:
global_model, metrics_df = federated_learning(Theta_data)


Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 25s 167ms/step - loss: 1.4047 - accuracy: 0.5035 - val_loss: 1.3741 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 23ms/step - loss: 1.3469 - accuracy: 0.5054 - val_loss: 1.3188 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 1s 31ms/step - loss: 1.2937 - accuracy: 0.5038 - val_loss: 1.2676 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 34ms/step - loss: 1.2445 - accuracy: 0.5078 - val_loss: 1.2204 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 1s 48ms/step - loss: 1.1991 - accuracy: 0.5005 - val_loss: 1.1769 - val_accuracy: 0.5151
Epoch 6/100
29/29 [==============================] - 1s 36ms/step - loss: 1.1570 - accuracy: 0.5035 - val_loss: 1.1368 - val_accuracy: 0.4849
Epoch 7/100
29/29 [==============================] - 1s 36ms/step - loss: 1.1179 - accuracy: 0.5256 - val_loss: 1.0997 - val_accuracy: 0.4849
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 9s 78ms/step - loss: 1.4057 - accuracy: 0.4929 - val_loss: 1.3755 - val_accuracy: 0.5045
Epoch 2/100
28/28 [==============================] - 0s 13ms/step - loss: 1.3496 - accuracy: 0.5014 - val_loss: 1.3218 - val_accuracy: 0.5045
Epoch 3/100
28/28 [==============================] - 0s 13ms/step - loss: 1.2982 - accuracy: 0.4992 - val_loss: 1.2723 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 0s 15ms/step - loss: 1.2503 - accuracy: 0.4853 - val_loss: 1.2264 - val_accuracy: 0.5045
Epoch 5/100
28/28 [==============================] - 0s 14ms/step - loss: 1.2059 - accuracy: 0.4935 - val_loss: 1.1838 - val_accuracy: 0.4955
Epoch 6/100
28/28 [==============================] - 0s 14ms/step - loss: 1.1647 - accuracy: 0.5011 - val_loss: 1.1443 - val_accuracy: 0.4955
Epoch 7/100
28/28 [==============================] - 0s 15ms/step - loss: 1.1265 - accuracy: 0.5113 - val_loss: 1.1076 - val_accuracy: 0.5045
Epoch 8/100
28/28 

...

29/29 [==============================] - 8s 53ms/step - loss: 0.0615 - accuracy: 0.9822 - val_loss: 0.6975 - val_accuracy: 0.5151
Epoch 2/100
29/29 [==============================] - 1s 21ms/step - loss: 0.0321 - accuracy: 0.9925 - val_loss: 0.6946 - val_accuracy: 0.5205
Epoch 3/100
29/29 [==============================] - 1s 22ms/step - loss: 0.0182 - accuracy: 0.9962 - val_loss: 0.6906 - val_accuracy: 0.5237
Epoch 4/100
29/29 [==============================] - 1s 22ms/step - loss: 0.0170 - accuracy: 0.9970 - val_loss: 0.6822 - val_accuracy: 0.6056
Epoch 5/100
29/29 [==============================] - 0s 13ms/step - loss: 0.0275 - accuracy: 0.9930 - val_loss: 0.6781 - val_accuracy: 0.5851
Epoch 6/100
29/29 [==============================] - 1s 26ms/step - loss: 0.0396 - accuracy: 0.9895 - val_loss: 0.6711 - val_accuracy: 0.6099
Epoch 7/100
29/29 [==============================] - 0s 13ms/step - loss: 0.0331 - accuracy: 0.9914 - val_loss: 0.6665 - val_accuracy: 0.6034
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 58ms/step - loss: 0.0805 - accuracy: 0.9751 - val_loss: 0.6976 - val_accuracy: 0.5057
Epoch 2/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0460 - accuracy: 0.9864 - val_loss: 0.6969 - val_accuracy: 0.5057
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 0.0158 - accuracy: 0.9980 - val_loss: 0.6944 - val_accuracy: 0.5283
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0166 - accuracy: 0.9972 - val_loss: 0.6897 - val_accuracy: 0.5543
Epoch 5/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0203 - accuracy: 0.9955 - val_loss: 0.6813 - val_accuracy: 0.5656
Epoch 6/100
28/28 [==============================] - 1s 25ms/step - loss: 0.0361 - accuracy: 0.9895 - val_loss: 0.6747 - val_accuracy: 0.6063
Epoch 7/100
28/28 [==============================] - 1s 22ms/step - loss: 0.0782 - accuracy: 0.9740 - val_loss: 0.6742 - val_accuracy: 0.6222
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 9s 72ms/step - loss: 0.1035 - accuracy: 0.9661 - val_loss: 0.6971 - val_accuracy: 0.5155
Epoch 2/100
31/31 [==============================] - 1s 24ms/step - loss: 0.0262 - accuracy: 0.9951 - val_loss: 0.6945 - val_accuracy: 0.5207
Epoch 3/100
31/31 [==============================] - 1s 22ms/step - loss: 0.0147 - accuracy: 0.9979 - val_loss: 0.6903 - val_accuracy: 0.5372
Epoch 4/100
31/31 [==============================] - 1s 23ms/step - loss: 0.0157 - accuracy: 0.9974 - val_loss: 0.6843 - val_accuracy: 0.5661
Epoch 5/100
31/31 [==============================] - 1s 23ms/step - loss: 0.0128 - accuracy: 0.9984 - val_loss: 0.6755 - val_accuracy: 0.5961
Epoch 6/100
31/31 [==============================] - 1s 22ms/step - loss: 0.0108 - accuracy: 0.9992 - val_loss: 0.6616 - val_accuracy: 0.6209
Epoch 7/100
31/31 [==============================] - 1s 23ms/step - loss: 0.0110 - accuracy: 0.9992 - val_loss: 0.6467 - val_accuracy: 0.6271
Epoch 8/100
31/31 

In [11]:

metrics_df.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.554,0.574,0.422,0.486,0.422,0.687,0.109
1,1,0.545,0.554,0.460,0.503,0.460,0.630,0.090
2,2,0.519,0.516,0.604,0.557,0.604,0.434,0.038
3,0,0.566,0.594,0.417,0.490,0.417,0.715,0.132
4,1,0.597,0.649,0.425,0.514,0.425,0.770,0.195
5,2,0.658,0.641,0.715,0.676,0.715,0.600,0.315
6,0,0.698,0.710,0.668,0.689,0.668,0.727,0.395
7,1,0.737,0.707,0.809,0.754,0.809,0.664,0.473
8,2,0.775,0.763,0.797,0.780,0.797,0.753,0.550
9,0,0.744,0.727,0.781,0.753,0.781,0.707,0.487


In [12]:
metrics_df.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_LSTM/Alpha_CNN_LSTM.csv', index = False)

# CNN

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(LSTM(256, return_sequences=True))
    # model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Alpha/CNN/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Alpha/CNN/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [16]:
global_model_CNN, metrics_df_CNN = federated_learning(Theta_data)

Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 0s 17ms/step - loss: 1.7629 - accuracy: 0.5458 - val_loss: 1.7603 - val_accuracy: 0.5409
Epoch 3/100
29/29 [==============================] - 1s 21ms/step - loss: 1.7522 - accuracy: 0.5477 - val_loss: 1.7511 - val_accuracy: 0.5496
Epoch 4/100
29/29 [==============================] - 0s 13ms/step - loss: 1.7419 - accuracy: 0.5568 - val_loss: 1.7419 - val_accuracy: 0.5345
Epoch 5/100
29/29 [==============================] - 1s 20ms/step - loss: 1.7313 - accuracy: 0.5471 - val_loss: 1.7328 - val_accuracy: 0.5582
Epoch 6/100
29/29 [==============================] - 0s 11ms/step - loss: 1.7212 - accuracy: 0.5652 - val_loss: 1.7240 - val_accuracy: 0.5269
Epoch 7/100
29/29 [==============================] - 0s 11ms/step - loss: 1.7122 - accuracy: 0.5509 - val_loss: 1.7148 - val_accuracy: 0.5216
Epoch 8/100
29/29 [==============================] - 0s 13ms/step - loss: 1.7009 - accuracy: 0.5754 - val_loss: 1.7059 - val_accuracy: 0.5205
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 18ms/step - loss: 1.7636 - accuracy: 0.5274 - val_loss: 1.7611 - val_accuracy: 0.5373
Epoch 3/100
28/28 [==============================] - 0s 13ms/step - loss: 1.7535 - accuracy: 0.5481 - val_loss: 1.7522 - val_accuracy: 0.5362
Epoch 4/100
28/28 [==============================] - 0s 10ms/step - loss: 1.7435 - accuracy: 0.5495 - val_loss: 1.7434 - val_accuracy: 0.5181
Epoch 5/100
28/28 [==============================] - 1s 18ms/step - loss: 1.7335 - accuracy: 0.5563 - val_loss: 1.7345 - val_accuracy: 0.5464
Epoch 6/100
28/28 [==============================] - 0s 10ms/step - loss: 1.7238 - accuracy: 0.5668 - val_loss: 1.7258 - val_accuracy: 0.5113
Epoch 7/100
28/28 [==============================] - 0s 10ms/step - loss: 1.7150 - accuracy: 0.5478 - val_loss: 1.7172 - val_accuracy: 0.4977
Epoch 8/100
28/28 [==============================] - 0s 10ms/step - loss: 1.7071 - accuracy: 0.5388 - val_loss: 1.7086 - val_accuracy: 0.5147
Epoch 9/100
28/28 

...

29/29 [==============================] - 0s 10ms/step - loss: 0.4797 - accuracy: 0.9165 - val_loss: 1.0698 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 10ms/step - loss: 0.4747 - accuracy: 0.9227 - val_loss: 1.0814 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 0s 11ms/step - loss: 0.4718 - accuracy: 0.9189 - val_loss: 1.0960 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 0s 10ms/step - loss: 0.4682 - accuracy: 0.9316 - val_loss: 1.1173 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 0s 11ms/step - loss: 0.4519 - accuracy: 0.9375 - val_loss: 1.1665 - val_accuracy: 0.4849
Epoch 7/100
29/29 [==============================] - 0s 16ms/step - loss: 0.4423 - accuracy: 0.9472 - val_loss: 1.1734 - val_accuracy: 0.4860
Epoch 8/100
29/29 [==============================] - 1s 18ms/step - loss: 0.4431 - accuracy: 0.9434 - val_loss: 1.2163 - val_accuracy: 0.4871
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 0s 11ms/step - loss: 0.4592 - accuracy: 0.9321 - val_loss: 1.0570 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 0s 11ms/step - loss: 0.4617 - accuracy: 0.9250 - val_loss: 1.0721 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 0s 11ms/step - loss: 0.4560 - accuracy: 0.9392 - val_loss: 1.0827 - val_accuracy: 0.4955
Epoch 5/100
28/28 [==============================] - 0s 10ms/step - loss: 0.4461 - accuracy: 0.9411 - val_loss: 1.1143 - val_accuracy: 0.4955
Epoch 6/100
28/28 [==============================] - 0s 10ms/step - loss: 0.4530 - accuracy: 0.9335 - val_loss: 1.1354 - val_accuracy: 0.4955
Epoch 7/100
28/28 [==============================] - 0s 11ms/step - loss: 0.4456 - accuracy: 0.9403 - val_loss: 1.1704 - val_accuracy: 0.4955
Epoch 8/100
28/28 [==============================] - 0s 16ms/step - loss: 0.4400 - accuracy: 0.9440 - val_loss: 1.1794 - val_accuracy: 0.4966
Epoch 9/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 0s 16ms/step - loss: 0.5088 - accuracy: 0.9034 - val_loss: 1.0830 - val_accuracy: 0.4855
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.4882 - accuracy: 0.9088 - val_loss: 1.0951 - val_accuracy: 0.4855
Epoch 4/100
31/31 [==============================] - 0s 14ms/step - loss: 0.4840 - accuracy: 0.9178 - val_loss: 1.1218 - val_accuracy: 0.4855
Epoch 5/100
31/31 [==============================] - 0s 13ms/step - loss: 0.4755 - accuracy: 0.9207 - val_loss: 1.1448 - val_accuracy: 0.4855
Epoch 6/100
31/31 [==============================] - 1s 21ms/step - loss: 0.4703 - accuracy: 0.9230 - val_loss: 1.1777 - val_accuracy: 0.4866
Epoch 7/100
31/31 [==============================] - 0s 14ms/step - loss: 0.4682 - accuracy: 0.9235 - val_loss: 1.2184 - val_accuracy: 0.4866
Epoch 8/100
31/31 [==============================] - 0s 15ms/step - loss: 0.4621 - accuracy: 0.9331 - val_loss: 1.2775 - val_accuracy: 0.4866
Epoch 9/100
31/31 

In [17]:
metrics_df_CNN.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.539,0.572,0.313,0.405,0.313,0.765,0.079
1,1,0.550,0.556,0.494,0.524,0.494,0.606,0.100
2,2,0.570,0.566,0.602,0.584,0.602,0.538,0.141
3,0,0.575,0.582,0.528,0.554,0.528,0.621,0.149
4,1,0.579,0.601,0.469,0.527,0.469,0.689,0.158
5,2,0.624,0.632,0.596,0.614,0.596,0.653,0.249
6,0,0.604,0.608,0.585,0.596,0.585,0.623,0.208
7,1,0.631,0.670,0.516,0.583,0.516,0.746,0.261
8,2,0.644,0.656,0.602,0.628,0.602,0.685,0.287
9,0,0.673,0.661,0.710,0.685,0.710,0.637,0.347


In [18]:
metrics_df_CNN.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN/Alpha_time_CNN.csv', index = False)

# GRU

In [19]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model_gru(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning_gru(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model_gru(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model_gru(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Alpha/CNN_GRU/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Alpha/CNN_GRU/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [20]:
global_model_gru, metrics_df_gru = federated_learning_gru(Theta_data)

Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 8s 69ms/step - loss: 1.4307 - accuracy: 0.5116 - val_loss: 1.4285 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 0s 13ms/step - loss: 1.4257 - accuracy: 0.5172 - val_loss: 1.4236 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 13ms/step - loss: 1.4207 - accuracy: 0.5210 - val_loss: 1.4189 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 1.4158 - accuracy: 0.5121 - val_loss: 1.4141 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 1.4107 - accuracy: 0.5307 - val_loss: 1.4093 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 0s 13ms/step - loss: 1.4063 - accuracy: 0.5216 - val_loss: 1.4046 - val_accuracy: 0.4849
Epoch 7/100
29/29 [==============================] - 1s 20ms/step - loss: 1.4013 - accuracy: 0.5242 - val_loss: 1.3999 - val_accuracy: 0.4871
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 72ms/step - loss: 1.4316 - accuracy: 0.4887 - val_loss: 1.4287 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 1s 19ms/step - loss: 1.4261 - accuracy: 0.5088 - val_loss: 1.4240 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 1s 18ms/step - loss: 1.4218 - accuracy: 0.5003 - val_loss: 1.4194 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 18ms/step - loss: 1.4168 - accuracy: 0.5122 - val_loss: 1.4148 - val_accuracy: 0.4955
Epoch 5/100
28/28 [==============================] - 0s 13ms/step - loss: 1.4121 - accuracy: 0.5105 - val_loss: 1.4102 - val_accuracy: 0.4955
Epoch 6/100
28/28 [==============================] - 0s 14ms/step - loss: 1.4078 - accuracy: 0.5091 - val_loss: 1.4057 - val_accuracy: 0.4955
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 1.4031 - accuracy: 0.5181 - val_loss: 1.4011 - val_accuracy: 0.4943
Epoch 8/100
28/28 

...

29/29 [==============================] - 7s 75ms/step - loss: 0.5261 - accuracy: 0.8518 - val_loss: 0.9188 - val_accuracy: 0.4860
Epoch 2/100
29/29 [==============================] - 1s 32ms/step - loss: 0.4507 - accuracy: 0.8812 - val_loss: 0.9133 - val_accuracy: 0.4892
Epoch 3/100
29/29 [==============================] - 1s 23ms/step - loss: 0.4347 - accuracy: 0.8909 - val_loss: 0.9071 - val_accuracy: 0.4968
Epoch 4/100
29/29 [==============================] - 1s 22ms/step - loss: 0.4526 - accuracy: 0.8793 - val_loss: 0.8885 - val_accuracy: 0.5075
Epoch 5/100
29/29 [==============================] - 1s 21ms/step - loss: 0.4236 - accuracy: 0.8920 - val_loss: 0.8838 - val_accuracy: 0.5162
Epoch 6/100
29/29 [==============================] - 1s 24ms/step - loss: 0.4432 - accuracy: 0.8922 - val_loss: 0.8846 - val_accuracy: 0.5205
Epoch 7/100
29/29 [==============================] - 1s 33ms/step - loss: 0.4256 - accuracy: 0.8939 - val_loss: 0.8772 - val_accuracy: 0.5377
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 7s 71ms/step - loss: 0.5308 - accuracy: 0.8483 - val_loss: 0.9151 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 1s 20ms/step - loss: 0.4428 - accuracy: 0.8953 - val_loss: 0.9058 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 1s 29ms/step - loss: 0.4341 - accuracy: 0.8899 - val_loss: 0.8862 - val_accuracy: 0.5000
Epoch 4/100
28/28 [==============================] - 1s 34ms/step - loss: 0.4331 - accuracy: 0.8919 - val_loss: 0.8594 - val_accuracy: 0.5498
Epoch 5/100
28/28 [==============================] - 1s 24ms/step - loss: 0.4490 - accuracy: 0.8854 - val_loss: 0.8561 - val_accuracy: 0.5554
Epoch 6/100
28/28 [==============================] - 1s 22ms/step - loss: 0.4421 - accuracy: 0.8902 - val_loss: 0.8588 - val_accuracy: 0.5588
Epoch 7/100
28/28 [==============================] - 0s 14ms/step - loss: 0.4239 - accuracy: 0.8962 - val_loss: 0.8652 - val_accuracy: 0.5588
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 7s 50ms/step - loss: 0.4949 - accuracy: 0.8659 - val_loss: 0.9216 - val_accuracy: 0.4855
Epoch 2/100
31/31 [==============================] - 1s 21ms/step - loss: 0.4869 - accuracy: 0.8690 - val_loss: 0.9102 - val_accuracy: 0.4897
Epoch 3/100
31/31 [==============================] - 1s 25ms/step - loss: 0.4618 - accuracy: 0.8765 - val_loss: 0.8829 - val_accuracy: 0.5248
Epoch 4/100
31/31 [==============================] - 1s 22ms/step - loss: 0.4877 - accuracy: 0.8594 - val_loss: 0.8869 - val_accuracy: 0.5258
Epoch 5/100
31/31 [==============================] - 1s 20ms/step - loss: 0.4622 - accuracy: 0.8775 - val_loss: 0.8819 - val_accuracy: 0.5455
Epoch 6/100
31/31 [==============================] - 1s 22ms/step - loss: 0.4629 - accuracy: 0.8698 - val_loss: 0.8666 - val_accuracy: 0.5496
Epoch 7/100
31/31 [==============================] - 1s 18ms/step - loss: 0.4524 - accuracy: 0.8868 - val_loss: 0.8630 - val_accuracy: 0.5496
Epoch 8/100
31/31 

In [21]:
metrics_df_gru

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.539363,0.542342,0.504188,0.522569,0.504188,0.574539,0.078727
1,1,0.560028,0.560801,0.553672,0.557214,0.553672,0.566384,0.120056
2,2,0.559237,0.555347,0.594378,0.574200,0.594378,0.524096,0.118474
3,0,0.578727,0.604444,0.455611,0.519580,0.455611,0.701843,0.157454
4,1,0.572740,0.568942,0.600282,0.584192,0.600282,0.545198,0.145480
5,2,0.594378,0.587361,0.634538,0.610039,0.634538,0.554217,0.188755
6,0,0.613903,0.619718,0.589615,0.604292,0.589615,0.638191,0.227806
7,1,0.615819,0.614525,0.621469,0.617978,0.621469,0.610169,0.231638
8,2,0.641566,0.643585,0.634538,0.639029,0.634538,0.648594,0.283133
9,0,0.677554,0.692029,0.639866,0.664926,0.639866,0.715243,0.355109


In [22]:
metrics_df_gru.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_GRU/Alpha_time_GRU.csv', index = False)